<a href="https://colab.research.google.com/github/IrynaBarnych/AI_23/blob/hw/module7/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [42]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [43]:
df.describe()

,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [44]:
!pip install -q surprise

In [45]:
!pip install -q optuna

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics
import lightgbm as lgb
import optuna


In [47]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["userId","blog_id","ratings"]], reader)

In [48]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [49]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using als...


In [50]:
from surprise import SVD

alfo = SVD(n_factors=100,
           n_epochs=50,
           )

algo.fit(trainset)

Estimating biases using als...


In [51]:
from surprise import KNNBasic

algo = KNNBasic(k=10,
                min_k=1,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [52]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 40)

algo.fit(trainset)


In [53]:
from surprise import BaselineOnly, accuracy


bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

Estimating biases using als...
mae = 1.6413346931007946
mse = 3.4403243582090783
rmse= 1.854811138151019
fcp = 0.4956943737423156


In [54]:
algo.predict(uid=2,
             iid=97)

Prediction(uid=2, iid=97, r_ui=None, est=3.5841576969934437, details={'was_impossible': False})

In [55]:
import optuna
from sklearn import metrics

def objective(trial):

    bsl_options = {
        'method': trial.suggest_categorical('method', ['als', 'sgd']),
        'reg_u': trial.suggest_loguniform('reg_u', 1e-10, 1e-2),
        'reg_i': trial.suggest_loguniform('reg_i', 1e-10, 1e-2)
    }


    model = BaselineOnly(bsl_options=bsl_options)


    model.fit(trainset)

    preds = model.test(testset)
    mae = accuracy.mae(preds, verbose=False)

    return mae

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)


study.best_params

[I 2024-05-21 20:08:24,562] A new study created in memory with name: no-name-58f56e56-e7ef-4ce3-ad26-2ec26bca5f78


Estimating biases using sgd...


<ipython-input-55-f9057487933f>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_u': trial.suggest_loguniform('reg_u', 1e-10, 1e-2),
<ipython-input-55-f9057487933f>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_i': trial.suggest_loguniform('reg_i', 1e-10, 1e-2)
[I 2024-05-21 20:08:26,577] Trial 0 finished with value: 1.6248644709174287 and parameters: {'method': 'sgd', 'reg_u': 3.831063592992656e-06, 'reg_i': 1.1985842251471654e-10}. Best is trial 0 with value: 1.6248644709174287.


Estimating biases using sgd...


[I 2024-05-21 20:08:28,227] Trial 1 finished with value: 1.6248644709174287 and parameters: {'method': 'sgd', 'reg_u': 8.2697929975512e-05, 'reg_i': 0.006109288983106719}. Best is trial 0 with value: 1.6248644709174287.


...

Estimating biases using sgd...


[I 2024-05-21 20:08:40,146] Trial 8 finished with value: 1.6248644709174287 and parameters: {'method': 'sgd', 'reg_u': 6.649720182204668e-08, 'reg_i': 1.2694478321879162e-08}. Best is trial 0 with value: 1.6248644709174287.


Estimating biases using sgd...


[I 2024-05-21 20:08:42,135] Trial 9 finished with value: 1.6248644709174287 and parameters: {'method': 'sgd', 'reg_u': 1.4029010054543365e-06, 'reg_i': 9.624598770102197e-08}. Best is trial 0 with value: 1.6248644709174287.


{'method': 'sgd',
 'reg_u': 3.831063592992656e-06,
 'reg_i': 1.1985842251471654e-10}

In [56]:
print(study.best_params)

{'method': 'sgd', 'reg_u': 3.831063592992656e-06, 'reg_i': 1.1985842251471654e-10}
